In [21]:
# All the libraries used on this project   
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import messagebox, ttk
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary

In [22]:
# Add the path of my Dataset
path = r'/home/mausterrr/Documents/Classes/Survey/SOM/House_Rent_Dataset.csv'
# Read it and save it on a Pandas DataFrame
df = pd.read_csv(path)


In [23]:
# Clean DataFrame
# Drop NaN values
df = df.dropna()
# Retain only selected columns
df= df[['Posted On','Rent', 'BHK', 'Bathroom', 'Size', 'Floor', 'Area Type', 'City', 'Furnishing Status']]
# Transform some strings into Integer for the floor number
# Created a function
def parse_floor(floor_str):
    #If the value is non existent con the column add a NaN
    if pd.isna(floor_str):
        return np.nan
    # Cleans white spaces at start or end of the string
    floor_str = floor_str.strip()
    # If value is Ground assign 0
    if "Ground" in floor_str:
        return 0
    # If value is upper basement assign -1
    elif "Upper Basement" in floor_str:
        return -1
    # If value is lower basement assign -2
    elif "Lower Basement" in floor_str:
        return -2
    # If value is out of, split it and leave it out
    elif "out of" in floor_str:
        try:
            return int(floor_str.split(" out of")[0])
        except:
            return np.nan
    else:
        # Capture single values
        try:
            return int(floor_str)
        except:
            return np.nan

# We apply the function to one of the columns
df["Floor"] = df["Floor"].apply(parse_floor)
# We print the info to check some values
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   Rent               4746 non-null   int64 
 2   BHK                4746 non-null   int64 
 3   Bathroom           4746 non-null   int64 
 4   Size               4746 non-null   int64 
 5   Floor              4746 non-null   int64 
 6   Area Type          4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
dtypes: int64(5), object(4)
memory usage: 333.8+ KB
None


# BIP problem

In [ ]:
# We create the model of the Binay Integer Programming Problem with ine parameter (client)
def BIP_model(client):
    # We use PuLP to create a maximization model
    model = LpProblem("Property_Selection", LpMaximize)
    # We create a list of binary variables, representing each property as as an "i" and a variable "x" that can be 1 or 0 depending on if it is chosen or not
    x = [LpVariable(f"x_{i}", cat=LpBinary) for i in range(len(df))]

    # Define the objective function, maximize the number of properties that satisfy all the constraints
    model += lpSum(x)

    # We start with the constraints or restrictions
    for i in range(len(df)):
        # If the rent surpasses the budget then it forces a 0 for the x(i)
        if df.iloc[i]["Rent"] > client["budget"]:
            model += x[i] == 0
        # If the size is smaller than the minimum acceptable size for the customer then it forces a 0 for the x(i) 
        if df.iloc[i]["Size"] < client["min_size"]:
            model += x[i] == 0
        # If the city is different from the selected by the customer it forces a 0 for the x(i)
        if client["city"] != "Any" and df.iloc[i]["City"] != client["city"]:
            model += x[i] == 0
        # If the area type is different from the selected by the customer it forces a 0 for the x(i)
        if client["area_type"] != "Any" and df.iloc[i]["Area Type"] != client["area_type"]:
            model += x[i] == 0
        # If the furnishing is different from the selected by the customer it forces a 0 for the x(i)
        if client["furnishing"] != "Any" and df.iloc[i]["Furnishing Status"] != client["furnishing"]:
            model += x[i] == 0
        # If the number of bathrooms is smaller from the indicated by the customer it forces a 0 for the x(i)
        if df.iloc[i]["Bathroom"] < client["bathrooms"]:
            model += x[i] == 0
        # If the floor is lower than the one selected by the customer it forces a 0 for the x(i)
        if df.iloc[i]["Floor"] < client["floors"]:
            model += x[i] == 0
        # If the number of bedrooms is lower than the customer preferences it forces a 0 for the x(i)
        if df.iloc[i]["BHK"] < client["bedrooms"]:
            model += x[i] == 0

    # Solves the model of Linear Programming
    model.solve()
    # Then it identifies all the values that stayed with a x(i) value of 1
    selected = [i for i in range(len(df)) if x[i].varValue == 1]
    # And returns them in a DataFrame
    return df.iloc[selected]

# User Interface to add all the information needed
def submit():
    client = {
        "budget": int(entry_budget.get()),
        "min_size": int(entry_size.get()),
        "city": city_var.get(),
        "area_type": area_var.get(),
        "furnishing": furnishing_var.get(),
        "bathrooms": int(entry_bath.get()),
        "floors": int(entry_floors.get()),
        "bedrooms": int(entry_bedrooms.get())
    }
    result = BIP_model(client)
    tree.delete(*tree.get_children())

    if result.empty:
        messagebox.showinfo("Results", "There are currently no properties that accommodate your needs.")
    else:
        result.to_csv("propiedades_optimas.csv", index=False)

        tree["columns"] = list(result.columns)
        tree["show"] = "headings"
        for col in result.columns:
            tree.heading(col, text=col)
            tree.column(col, width=100, anchor="center")

        for _, row in result.iterrows():
            tree.insert("", "end", values=list(row))

        messagebox.showinfo("Results", f"{len(result)} properties were selected.\nSaved in propiedades_optimas.csv")

    # Sensitivity Analysis Section
    variations = [-0.1, -0.05, 0, 0.05, 0.1]
    sensitivity_data = []

    for delta in variations:
        mod_budget = client["budget"] * (1 + delta)
        mod_client = client.copy()
        mod_client["budget"] = mod_budget
        mod_result = BIP_model(mod_client)
        sensitivity_data.append({
            "Budget_Adjustment": f"{int(delta * 100)}%",
            "Adjusted_Budget": int(mod_budget),
            "Properties_Found": len(mod_result)
        })

    sens_df = pd.DataFrame(sensitivity_data)
    sens_df.to_csv("sensibilidad_presupuesto.csv", index=False)

    # Create popup window for sensitivity results
    sens_window = tk.Toplevel(root)
    sens_window.title("Budget Sensitivity Analysis")

    sens_tree = ttk.Treeview(sens_window)
    sens_tree.pack(fill="both", expand=True)

    sens_tree["columns"] = list(sens_df.columns)
    sens_tree["show"] = "headings"
    for col in sens_df.columns:
        sens_tree.heading(col, text=col)
        sens_tree.column(col, width=120, anchor="center")

    for _, row in sens_df.iterrows():
        sens_tree.insert("", "end", values=list(row))

root = tk.Tk()
root.title("Client Simulator")

tk.Label(root, text="Monthly Budget:").pack()
entry_budget = tk.Entry(root)
entry_budget.pack()

tk.Label(root, text="Min. Size (sqft):").pack()
entry_size = tk.Entry(root)
entry_size.pack()

tk.Label(root, text="Preferred City:").pack()
city_var = tk.StringVar(root)
city_var.set("Any")
tk.OptionMenu(root, city_var, "Any", *df["City"].unique()).pack()

tk.Label(root, text="Area Type:").pack()
area_var = tk.StringVar(root)
area_var.set("Any")
tk.OptionMenu(root, area_var, "Any", *df["Area Type"].unique()).pack()

tk.Label(root, text="Furnishing Status:").pack()
furnishing_var = tk.StringVar(root)
furnishing_var.set("Any")
tk.OptionMenu(root, furnishing_var, "Any", *df["Furnishing Status"].unique()).pack()

tk.Label(root, text="Minimum Bathrooms:").pack()
entry_bath = tk.Entry(root)
entry_bath.pack()

tk.Label(root, text="Minimum Floors:").pack()
entry_floors = tk.Entry(root)
entry_floors.pack()

tk.Label(root, text="Minimum Bedrooms:").pack()
entry_bedrooms = tk.Entry(root)
entry_bedrooms.pack()

tk.Button(root, text="Search Properties", command=submit).pack(pady=10)

frame_results = tk.Frame(root)
frame_results.pack(fill="both", expand=True)

tree = ttk.Treeview(frame_results)
tree.pack(fill="both", expand=True)

scrollbar_y = ttk.Scrollbar(frame_results, orient="vertical", command=tree.yview)
scrollbar_y.pack(side="right", fill="y")
tree.configure(yscrollcommand=scrollbar_y.set)

scrollbar_x = ttk.Scrollbar(frame_results, orient="horizontal", command=tree.xview)
scrollbar_x.pack(side="bottom", fill="x")
tree.configure(xscrollcommand=scrollbar_x.set)

style = ttk.Style()
style.theme_use("clam")
style.configure("Treeview", rowheight=25)

root.mainloop()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mausterrr/Documents/Classes/Survey/SOM/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/97f1cdb72d3e4dc2965b5e06a784e88b-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/97f1cdb72d3e4dc2965b5e06a784e88b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4620 COLUMNS
At line 23474 RHS
At line 28090 BOUNDS
At line 32837 ENDATA
Problem MODEL has 4615 rows, 4746 columns and 4615 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 164 - 0.00 seconds
Cgl0002I 4582 variables fixed
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -164 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were 